In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
%matplotlib inline

---

In [126]:
#DBM data below

In [127]:
DBM = pd.DataFrame()
for file in glob.glob("DBM*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    DBM = DBM.append(df, ignore_index=True)

In [128]:
DBM = DBM.loc[:, ["Month", "Advertiser", "Public Inventory", "Impressions", 
                  "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency"]]

In [129]:
DBM.columns = ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency"]

In [130]:
DBM["DSP"] = "DBM"
DBM["Type"] = "General"

---

In [131]:
#YouTube/TrueView data below

In [132]:
YT = pd.DataFrame()
for file in glob.glob("YT*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    YT = YT.append(df, ignore_index=True)

In [133]:
YT["Public"] = "N/A"
YT["DSP"] = "DBM"
YT["Type"] = "TrueView"

In [134]:
YT = YT.loc[:, ["Month", "Advertiser", "Public","Impressions", 
                "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency", "DSP", "Type"]]

In [135]:
YT.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks',
       'Spend', 'Media', 'Currency', 'DSP', 'Type']

---

In [136]:
#Videology data below
#please note the data in the template the time frame please select the specific time rather than 'last month'

In [137]:
videology = pd.DataFrame()
for file in glob.glob("Videology*.csv"):
    df = pd.read_csv(file)
    df = df[df["Placement ID"].isnull() == False]
    videology = videology.append(df, ignore_index=True)

In [138]:
videology.columns = ["Advertiser", "Placement", "PlacementName", "Date", "Impressions", "Spend", "Clicks"]

In [139]:
list1=[]
for month in videology.Date:
    g = month[-4:] + "/" + "{0:0=2d}".format(time.strptime(month[:3], "%b").tm_mon)
    list1.append(g)

In [140]:
#repmoving the thousand separator 
videology.Impressions = videology.Impressions.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Clicks = videology.Clicks.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Spend = videology.Spend.apply(lambda x: float(x.split()[0].replace(',', '')))

In [141]:
videology["Currency"] = "GBP"
videology["DSP"] = "Videology"
videology["Type"] = "General"
videology.loc[(videology.PlacementName.str.contains("Channel 4")) | 
              (videology.PlacementName.str.contains("C4")), "Type"] = "Channel 4"
videology["Media"] = videology.Spend * 0.7
videology.loc[videology.Type == "Channel 4", "Media"] = videology.Spend * 0.8
videology["Public"] = "N/A"
videology["Month"] = list1

In [142]:
videology = videology.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend',
       'Media', 'Currency', 'DSP', 'Type']]

---

In [143]:
#TheTradeDesk data below

In [144]:
TTD = pd.DataFrame()
for file in glob.glob("TTTD*.csv"):
    df=pd.read_csv(file)
    TTD = TTD.append(df, ignore_index=True)

In [145]:
TTD["Month"] = TTD.Date.str.split("/").str[2] + "/" + TTD.Date.str.split("/").str[1]
TTD["Public"] = "Yes"
TTD.loc[TTD["Inventory Contract"].isnull() != True, "Public"] = "No"
TTD["DSP"] = "TTD"
TTD["Type"] = "Genral"

In [146]:
TTD.columns = ['Date', 'Advertiser', 'Currency', 'Inventory Contract','Impressions', 'Clicks', 'Spend',
       'Media', 'Month', 'Public', 'DSP', 'Type']

In [147]:
TTD = TTD.loc[:, ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency", "DSP", "Type"]]

In [148]:
TTD.Spend = TTD.Spend.str.replace(",", "")
TTD.Spend = pd.to_numeric(TTD.Spend) / 100000000000000
TTD.Media = TTD.Media.str.replace(",", "")
TTD.Media = pd.to_numeric(TTD.Media) / 100000000000000

---

In [149]:
#RocketFuel data below

In [150]:
RF = pd.DataFrame()
for file in glob.glob("RF*.csv"):
    df = pd.read_csv(file, header=1)
    RF = RF.append(df, ignore_index=True)
RF.Spend = RF.Spend.str.replace("GBP ", "")
RF["Media Cost"] = RF["Media Cost"].str.replace("GBP ", "")

In [151]:
RF["Public"] = "Yes"
RF.loc[RF["Exchange Deal ID"].isnull() == False, "Public"] = "No"
RF["Month"] = RF["Year and Month"].str.split("-").str[0] + "/" + RF["Year and Month"].str.split("-").str[1]
RF["DSP"] = "RocketFuel"
RF["Type"] = "General"
RF["Currency"] = "GBP"

In [152]:
RF = RF.loc[:, ["Month", "Advertiser", "Public", "Imps", "Clicks", "Spend", "Media Cost", "Currency", "DSP", "Type"]]

In [153]:
RF.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend', 'Media', 'Currency', 'DSP', 'Type']

In [154]:
RF.Spend = RF.Spend.astype("float64")
RF.Media = RF.Media.astype("float64")

---

In [155]:
#Amazon data below

In [156]:
AAP_old = pd.read_csv("Amazon_0116_0317.csv")

In [157]:
AAP_old = AAP_old.head(147)

In [158]:
AAP_old.date = AAP_old.date.str.split(" ").str[0]
AAP_old.date = AAP_old.date.str.split("/").str[2] + "/" + AAP_old.date.str.split("/").str[1]

In [159]:
AAP_old.columns = ['Month', 'Advertiser', 'advertiser_id', 'Impressions', 'Clicks',
       'Media', 'net cost', 'agency markup', 'Spend']

In [160]:
AAP = pd.DataFrame()
for file in glob.glob("AAP*.csv"):
    df = pd.read_csv(file, usecols=["Date", "Advertiser", "Impressions", "Clickthroughs", 
                                    "Total Cost", "Supply Cost", "Order Currency"])
    AAP = AAP.append(df, ignore_index=True) 

In [161]:
AAP = AAP.drop(AAP.index[AAP["Total Cost"] == 0]).reset_index(drop=True)

In [162]:
list2=[]
for month in AAP.Date:
    g = month[-4:] + "/" + "{0:0=2d}".format(time.strptime(month[:3], "%b").tm_mon)
    list2.append(g)

In [163]:
AAP.Date = list2
AAP["Public"] = "N/A"
AAP["DSP"] = "AAP"
AAP["Type"] = "General"

In [164]:
AAP.columns = ['Month', 'Advertiser', 'Currency', 'Media', 'Spend','Impressions', 'Clicks', 'Public', 'DSP', 'Type']

In [165]:
AAP = AAP.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend', 'Media', 'Currency', 'DSP', 'Type']]

In [166]:
AAP.Spend = AAP.Spend.convert_objects(convert_numeric=True)

C:\Users\ylin\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


---

In [167]:
#Turn data below

In [168]:
TURN_old = pd.read_csv("oldTURN_0317_0417.csv")

In [169]:
TURN_old.Month = TURN_old.Month.str.split("/").str[2] + "/" + TURN_old.Month.str.split("/").str[0]

In [170]:
TURN_old["Public"] = "Yes"
TURN_old.loc[~TURN_old.Deal_ID.isin([-1]), "Public"] = "No"
TURN_old["DSP"] = "TURN"
TURN_old["Type"] = "General"
TURN_old["Media"] = TURN_old.Spend * 0.7

In [171]:
TURN_old = TURN_old.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions','Clicks','Spend','Media','Currency', 'DSP', 'Type']]

In [172]:
TURN = pd.DataFrame()
for file in glob.glob("TURN*.csv"):
    df = pd.read_csv(file, encoding="ISO-8859-1")
    TURN = TURN.append(df, ignore_index=True)

In [173]:
TURN.columns = ['Month', 'Advertiser', 'deal_id', 'deal_name', 'Impressions', 'Clicks', 'Currency', 'Spend']

In [174]:
TURN.Spend = TURN.Spend.str.replace("£", "")
TURN.Spend = TURN.Spend.str.replace(",", "")
TURN.Spend = pd.to_numeric(TURN.Spend)

In [175]:
TURN.Month = TURN.Month.str.split("_").str[0] + "/" + TURN.Month.str.split("_").str[1]

In [176]:
TURN["Public"] = "Yes"
TURN.loc[~(TURN.deal_id == "Unknown"), "Public"] = "No"
TURN["DSP"] = "TURN"
TURN["Type"] = "General"
TURN["Media"] = TURN.Spend * 0.7

In [177]:
TURN = TURN.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks','Spend','Media','Currency','DSP', 'Type']]
TURN = TURN.append(TURN_old, ignore_index=True)

---

In [178]:
#Data from all DSPs

In [179]:
data = DBM.append([TTD, YT, videology, RF, AAP, TURN], ignore_index=True)

In [180]:
np.unique(data.Currency)
USDtoGBP = 0.77
EURtoGBP = 0.84

In [181]:
data["cSpend"] = data.Spend 
data.loc[data.Currency == "USD","cSpend"] = data.Spend * USDtoGBP
data.loc[data.Currency == "EUR","cSpend"] = data.Spend * EURtoGBP

---

In [182]:
#file to ZO folder on desktop

In [183]:
data.sort_values("Month", ascending=True).to_csv("C:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend.csv", index=False)

In [184]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], 
                 aggfunc=np.sum, margins=True).round().to_csv("C:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend pivot.csv")

In [185]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], aggfunc=np.sum, margins=True).round()

DSP          AAP         DBM                        RocketFuel       TTD  \
Type     General     General               TrueView    General    Genral   
Public       N/A          No         Yes        N/A        Yes        No   
Month                                                                      
2016/01      NaN    462491.0   2015535.0   126240.0        NaN       NaN   
2016/02      NaN    599799.0   2321375.0   180716.0        NaN       NaN   
2016/03      NaN   1042068.0   2574432.0   279131.0        NaN       NaN   
2016/04      NaN    525837.0   2141692.0   371005.0        NaN       NaN   
2016/05      NaN    537853.0   1718345.0   153631.0        NaN       NaN   
2016/06      NaN    601384.0   1900652.0   286835.0        NaN       NaN   
2016/07      NaN    470456.0   1444104.0   225570.0        NaN   31395.0   
2016/08      NaN    685058.0   1577548.0   138528.0        NaN   90540.0   
2016/09      NaN    843696.0   1788445.0   172688.0        NaN   63959.0   
2016/10      NaN    784882.0   2080459.0   380387.0        NaN      34.0   
2016/11      NaN    987119.0   1905395.0   350509.0        NaN   29214.0   
2016/12      NaN   1018084.0   1783496.0   174375.0        NaN   80587.0   
2017/01      NaN    442769.0   1138666.0   201930.0     5743.0   16078.0   
2017/02      NaN    419031.0    947721.0   175352.0     6835.0   15637.0   
2017/03      NaN    596981.0   1165207.0   139000.0     5570.0   45042.0   
2017/04  25278.0    593276.0   1188412.0   132948.0    22024.0   21149.0   
2017/05  20469.0    298205.0    926253.0    63504.0    31687.0   56710.0   
2017/06  30055.0    351915.0    961273.0    85682.0        1.0   79248.0   
2017/07  21559.0    271127.0    785516.0   120438.0     3704.0   74340.0   
All      97361.0  11532029.0  30364525.0  3758471.0    75565.0  603933.0   

DSP                  TURN           Videology                    All  
Type              General           Channel 4    General              
Public        Yes      No      Yes        N/A        N/A              
Month                                                                 
2016/01       0.0     NaN      NaN        NaN        NaN   2604266.0  
2016/02       0.0     NaN      NaN        NaN        NaN   3101890.0  
2016/03       NaN     NaN      NaN        NaN        NaN   3895631.0  
2016/04    1974.0     NaN      NaN        NaN        NaN   3040509.0  
2016/05   10261.0     NaN      NaN   121445.0   341517.0   2883051.0  
2016/06   19950.0     NaN      NaN   163329.0   353411.0   3325562.0  
2016/07   13668.0     NaN      NaN   104554.0   162814.0   2452561.0  
2016/08   32341.0     NaN      NaN   215635.0   113612.0   2853263.0  
2016/09   13476.0     NaN      NaN   302641.0   161261.0   3346166.0  
2016/10   27429.0     NaN      NaN   831893.0   227547.0   4332630.0  
2016/11   26356.0     NaN      NaN   841687.0   145597.0   4285877.0  
2016/12   28732.0     NaN      NaN   480049.0    31477.0   3596800.0  
2017/01    8943.0     NaN      NaN   264591.0   212558.0   2291277.0  
2017/02   27589.0     NaN      NaN   518240.0   228212.0   2338618.0  
2017/03   62396.0     0.0  14250.0   433708.0   237404.0   2699557.0  
2017/04   50544.0     NaN      1.0   518357.0   147026.0   2699015.0  
2017/05   45494.0     2.0   7860.0   483277.0    97798.0   2031258.0  
2017/06   31904.0  2204.0  20437.0   345388.0    89350.0   1997456.0  
2017/07   23412.0     NaN      NaN   350381.0    90070.0   1740547.0  
All      424470.0  2206.0  42547.0  5975174.0  2639654.0  55515935.0

In [186]:
data.pivot_table(values="cSpend", index="Advertiser", aggfunc=np.sum).sort_values("cSpend", ascending=False)

,cSpend
Advertiser,
Next_UK_Zenith,9.816020e+06
Natwest (DFA)_UK_Zenith,4.355141e+06
Mercedes-Benz_UK_Zenith/Ninah,3.923772e+06
Toyota_UK_Zenith,3.373294e+06
Entertainment One_UK_Zenith,2.776404e+06
Aviva_UK_Zenith,2.466817e+06
Christies_UK_Zenith International,2.148874e+06
Mercedes UK,1.847864e+06
Delta Airline_UK_Zenith International,1.191990e+06
